**Task 3 — Reproduce practice in lecture using LCEL**

*A chain is a sequence of steps or model calls connected together to achieve a larger task. Each step can involve retrieving information, transforming text, or invoking a language model in some way, and then passing its output on to the next step in the chain. This structure helps you build more complex workflows or pipelines using multiple actions in a simple, organized manner.*

### 1. Setup

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool
import json

# ============================================================
# Helper functions for messages
# ============================================================
def assistant_message(content):
    """Create an assistant role message."""
    return ("assistant", content)

def system_message(content):
    """Create a system role message."""
    return ("system", content)

def user_message(content):
    """Create a user role message."""
    return ("user", content)


# ============================================================
# Conversation chain factory
# ============================================================
def create_conversation_chain(message_sequence):
    """
    Create a LangChain expression pipeline from a list of role-based messages.

    Parameters:
        message_sequence (list): Sequence of (role, content) tuples

    Returns:
        Runnable: A chain that formats the prompt and sends it to the model
    """
    # Example: Using ChatOllama here; swap for ChatOpenAI if needed
    from langchain_ollama import ChatOllama
    llama3_chat_model = ChatOllama(
        model="llama3",
        temperature=0,
        tools=list(tool_registry.values())  # Ignored by Ollama, but kept for API consistency
    )

    prompt_template = ChatPromptTemplate.from_messages(message_sequence)
    return prompt_template | llama3_chat_model

### 2. Basic Prompting

In [ ]:
# Prompt template asking for the capital of a given country
capital_query_prompt = [user_message("What is the capital of {country}?")]

# Build the LCEL chain from the prompt
capital_lookup_chain = create_conversation_chain(capital_query_prompt)

# List of countries to query
countries_to_lookup = ["France", "Germany"]

# Loop over countries and print their capitals
for country_name in countries_to_lookup:
    capital_result = capital_lookup_chain.invoke({"country": country_name})
    print(capital_result.content)


The capital of France is Paris.
The capital of Germany is Berlin.


### 3. Summarization

In [ ]:
# Prompt template to summarize a passage in 2–3 sentences
summary_prompt = [user_message(
    "Summarize the following text:\n{passage}"
)]

# List of passages to summarize
passages_to_summarize = [
    """
    Artificial intelligence (AI) refers to the simulation of human intelligence in machines 
    that are programmed to think and learn. It has applications in various fields, 
    including healthcare, finance, and transportation.
    """,
    """
    In the face of adversity, resilience isn't just about bouncing back—it's about transforming. 
    Like roots pushing through stone, growth often happens in resistance. Each challenge becomes a teacher; 
    each setback, a foundation for reinvention. Whether it's in personal loss, career disruption, or even global turmoil, 
    those who rise aren't untouched by hardship—they are shaped by it. They emerge with deeper empathy, sharper purpose, 
    and wider perspective. True resilience is quiet, steady, and often invisible until the bloom appears. And when it does, 
    it speaks of strength not just endured—but chosen.
    """
]

# Build the summarization chain
summarization_chain = create_conversation_chain(summary_prompt)

# Generate and display summaries
for passage in passages_to_summarize:
    summary_result = summarization_chain.invoke({"passage": passage})
    print(summary_result.content, "\n")

Here is a summary of the text in 2-3 sentences:

Artificial intelligence (AI) is the ability of machines to mimic human intelligence by thinking and learning. AI has numerous practical applications across various industries, such as healthcare, finance, and transportation. Its capabilities allow it to perform tasks that would typically require human intelligence. 

Here is a summary of the text in 2-3 sentences:

Resilience involves not just bouncing back from adversity, but also transforming and growing as a result of the challenges faced. Each obstacle serves as a teacher and foundation for reinvention, shaping individuals into stronger, more empathetic, and purposeful versions of themselves. True resilience is quiet, steady, and often invisible until its effects are seen in the bloom of personal growth and strength. 



### 4. Information Extraction

In [ ]:
# Build the user instruction message template
extraction_prompt = [user_message((
    "Extract the {field_to_extract} from the following text:\n"
    "John Doe, a 29-year-old software engineer from San Francisco, recently joined OpenAI as a research scientist."
))]

# List of data fields we want to extract
fields_to_extract = ["name", "occupation", "age", "location"]

# Build the LCEL chain from the prompt
extraction_chain = create_conversation_chain(extraction_prompt)

# Loop over each field and extract it
for field in fields_to_extract:
    result = extraction_chain.invoke({"field_to_extract": field})
    print(result.content, "\n")

The name mentioned in the text is:

* John Doe 

The occupation mentioned is:

1. Software Engineer
2. Research Scientist 

The age mentioned in the text is:

29 

The location mentioned in the text is:

* San Francisco 



### 5. Transformation

In [77]:
# Prompt template to translate a given text into a target language
translation_prompt_template = [
    user_message("Translate the following text to {target_language}:\n{source_text}")
]

# List of translation tasks with source text and target language
translation_tasks = [
    {
        "source_text": "The weather is nice today.",
        "target_language": "French"
    },
    {
        "source_text": "I am learning how to use LangChain with Ollama.",
        "target_language": "Spanish"
    }
]

# Build the translation chain
translation_chain = create_conversation_chain(translation_prompt_template)

# Execute translations
for task in translation_tasks:
    translation_result = translation_chain.invoke(task)
    print(translation_result.content, "\n")

Le temps est agréable aujourd'hui.

(Note: "nice" can also be translated as "agréable", but if you want to use a more formal tone, you could say "le ciel est ensoleillé aujourd'hui" which means "the sky is sunny today".) 

Estoy aprendiendo a utilizar LangChain con Ollama.

Note: "LangChain" is likely referring to LangChain AI, a language model developed by Meta AI. If you meant something else, please let me know and I'll be happy to help! 



### 6. Expansion

In [ ]:
# List of creative writing prompts
creative_writing_tasks = [
    user_message("Write a short story about a dragon who learns to code."),
    user_message("Write a poem about a robot exploring space.")
]

# Loop through each creative writing prompt
for role_prompt in creative_writing_tasks:
    # Build a conversation chain for the current task
    creative_writing_chain = create_conversation_chain(role_prompt)
    
    # Generate the model's response
    writing_result = creative_writing_chain.invoke({})
    
    # Display the generated content
    print(role_prompt[1], "\n")
    print(writing_result.content, "\n\n\n")

Write a short story about a dragon who learns to code. 

In the heart of the mystical forest, a magnificent dragon named Ember lived a life of solitude and curiosity. While his fellow dragons spent their days hoarding treasure and breathing fire, Ember was fascinated by the strange, glowing rectangles that humans used to communicate.

One day, while exploring the outskirts of the forest, Ember stumbled upon a group of programmers working on a new project. The dragon's eyes widened as he watched them typing away on their computers, creating something that seemed like magic to him.

Intrigued, Ember approached the group and introduced himself in his best human-like tone (which was still quite draconic). The programmers, startled by the sudden appearance of a giant fire-breathing creature, hesitated before welcoming Ember with open arms.

As they showed him their work, Ember became captivated by the world of coding. He watched as lines of code transformed into functional programs, and his

### 7. Role-based Prompting

In [85]:
# List of role-based instruction prompts for the assistant
role_based_instructions = [
    user_message("As a professional chef, explain how to make a perfect omelette."),
    user_message("Explain the following topic as if you were a kindergarten teacher, using simple words and fun examples:\n\nQuantum physics")
]

# Loop through each role-based instruction
for role_instruction in role_based_instructions:
    # Build a conversation chain for the current role-based task
    role_instruction_chain = create_conversation_chain(role_instruction)
    
    # Generate the model's response
    role_response = role_instruction_chain.invoke({})
    
    # Display the original instruction and the generated content
    print(role_instruction[1], "\n")
    print(role_response.content, "\n\n\n")

As a professional chef, explain how to make a perfect omelette. 

The humble omelette! It's a staple in many kitchens around the world, and yet, it can be surprisingly tricky to get just right. As a professional chef, I'm happy to share my secrets for making the perfect omelette.

**Step 1: Choose the Right Pan**
Before we even start cooking, let's talk about the pan. You'll want a non-stick skillet or omelette pan that's heat-resistant and has a smooth surface. A well-seasoned cast-iron skillet works beautifully too! Avoid using a stainless steel or aluminum pan, as they can react with the eggs and cause them to stick.

**Step 2: Crack Those Eggs**
Fresh eggs are essential for a perfect omelette. I like to use large or extra-large eggs for this recipe. Crack 2-3 eggs (depending on the size you prefer) into a bowl and whisk them together with a fork until they're just combined. Don't overbeat! You want to maintain some of that lovely egg texture.

**Step 3: Heat Up the Pan**
Preheat yo

### 8. Few-shot Prompting

In [88]:
# Prompt template containing English-to-French translation examples
french_translation_prompt_template = [
    user_message(
        """Translate the following English phrases to French:

        English: Hello
        French: Bonjour

        English: Thank you
        French: Merci

        English: Good night
        French:
        """
                )
]

# Build the translation conversation chain
french_translation_chain = create_conversation_chain(french_translation_prompt_template)

# Invoke the chain to get the translation
french_translation_output = french_translation_chain.invoke({})

# Display the translation result
print(french_translation_output.content, "\n")

Here are the translations:

English: Hello
French: Bonjour (as you mentioned)

English: Thank you
French: Merci (as you mentioned)

English: Good night
French: Bonne nuit 



### 9. Chain-of-Thought Prompting

In [89]:
# List of problem-solving prompts for the assistant
problem_solving_prompts = [
    user_message(
        "If it takes 5 machines 5 minutes to make 5 widgets, "
        "how long would it take 100 machines to make 100 widgets? Explain your reasoning."
    ),
    user_message(
        "A rectangle has a perimeter of 50 cm. Its length is 5 cm longer than its width. "
        "Find the length and width. Show your work step by step."
    )
]

# Loop through each problem-solving prompt
for problem_prompt in problem_solving_prompts:
    # Build a conversation chain for the current problem
    problem_solving_chain = create_conversation_chain(problem_prompt)
    
    # Generate the assistant's response
    problem_response = problem_solving_chain.invoke({})
    
    # Display the original instruction and the generated content
    print(problem_prompt[1], "\n")
    print(problem_response.content, "\n\n\n")

If it takes 5 machines 5 minutes to make 5 widgets, how long would it take 100 machines to make 100 widgets? Explain your reasoning. 

A classic lateral thinking puzzle!

Let's break it down:

* It takes 5 machines 5 minutes to make 5 widgets.
* This means that each machine takes 5 minutes to make 1 widget.

Now, if we have 100 machines, each machine will still take the same amount of time to make 1 widget: 5 minutes.

So, it would take 100 machines 5 minutes to make 100 widgets. The number of machines doesn't affect the time it takes for each machine to complete its task.

Therefore, the answer is: 5 minutes! 



A rectangle has a perimeter of 50 cm. Its length is 5 cm longer than its width. Find the length and width. Show your work step by step. 

Let's break it down step by step!

Let the width be x cm.

Since the length is 5 cm longer than the width, the length can be expressed as x + 5 cm.

The perimeter of a rectangle is the sum of all its sides. Since there are two lengths and t

### 10. System Prompts

In [92]:
# Function to build a role-based conversation prompt sequence
def build_tone_specific_prompt(system_tone_instruction, user_question):
    """Return a list containing the system's tone/persona instruction and the user question."""
    return [system_message(system_tone_instruction), user_message(user_question)]

# List of different system tones/personalities for the assistant
assistant_tone_instructions = [
    "You are a helpful assistant that provides concise and accurate information.",
    "You're a witty assistant who explains things with clever analogies and playful sarcasm."
]

# The question we want the assistant to answer in different tones
privacy_question = "Can you explain the importance of data privacy?"

# Loop through each tone and generate the assistant's response
for tone_instruction in assistant_tone_instructions:
    tone_specific_prompt = build_tone_specific_prompt(tone_instruction, privacy_question)
    tone_specific_chain = create_conversation_chain(tone_specific_prompt)
    tone_specific_response = tone_specific_chain.invoke({})
    
    # Display the tone/persona and the generated content
    print(tone_instruction, "\n")
    print(tone_specific_response.content, "\n\n\n")


You are a helpful assistant that provides concise and accurate information. 

Data privacy is crucial in today's digital age because it protects individuals' personal and sensitive information from unauthorized access, use, or disclosure. Here are some reasons why data privacy is important:

1. **Protection of Personal Information**: Data privacy ensures that your personal information, such as name, address, phone number, and financial details, remains confidential and secure.
2. **Prevention of Identity Theft**: By keeping your data private, you reduce the risk of identity theft, where criminals use stolen information to impersonate you or access your accounts.
3. **Preservation of Trust**: When organizations handle personal data responsibly, individuals are more likely to trust them with their information, fostering a sense of security and loyalty.
4. **Compliance with Regulations**: Data privacy laws, such as the General Data Protection Regulation (GDPR) in the EU or the California 

### 11. Utilized prompt

In [99]:
# Function to construct a conversation with history context
def build_history_conversation_prompt(system_role_text, first_user_question, assistant_initial_answer, second_user_question):
    """Return a list representing a short role-based historical conversation."""
    return [
        system_message(system_role_text),        # <- calls the helper function
        user_message(first_user_question),
        assistant_message(assistant_initial_answer),
        user_message(second_user_question)
    ]

# Role definition and conversation turns
history_system_text = "You are a helpful assistant knowledgeable in history."
initial_history_question = "Who was the first president of the United States?"
initial_history_answer = "George Washington was the first president of the United States."
follow_up_history_question = "When did he take office?"

# Build the role-based prompt
history_conversation_prompt = build_history_conversation_prompt(
    history_system_text,
    initial_history_question,
    initial_history_answer,
    follow_up_history_question
)

# Create and run the conversation chain
history_conversation_chain = create_conversation_chain(history_conversation_prompt)
history_response = history_conversation_chain.invoke({})

# Display the assistant's final response
print(history_response.content, "\n")


George Washington took office as the first President of the United States on April 30, 1789. He was inaugurated for a second term on March 4, 1793, and served until March 4, 1797. 



### 12. Creating an AI Agent

In [44]:
# ============================================================
# Define arithmetic tools
# ============================================================
# Each function is decorated with @tool so LangChain can register it
# and make it available for function calling or tool invocation.

@tool
def add_numbers(a: float, b: float) -> float:
    """
    Return the sum of two numbers.

    Parameters:
        a (float): First number
        b (float): Second number

    Returns:
        float: The sum of a and b
    """
    # Perform addition and return the result
    return a + b


@tool
def subtract_numbers(a: float, b: float) -> float:
    """
    Return the difference between two numbers.

    Parameters:
        a (float): First number
        b (float): Second number

    Returns:
        float: The result of a - b
    """
    # Perform subtraction and return the result
    return a - b


@tool
def multiply_numbers(a: float, b: float) -> float:
    """
    Return the product of two numbers.

    Parameters:
        a (float): First number
        b (float): Second number

    Returns:
        float: The result of a * b
    """
    # Perform multiplication and return the result
    return a * b


# ============================================================
# Tool registry
# ============================================================
# Maps tool names (as strings) to their corresponding @tool objects.
# This allows dynamic lookup and execution by name.

tool_registry = {
    "add_numbers": add_numbers,
    "subtract_numbers": subtract_numbers,
    "multiply_numbers": multiply_numbers
}


# ============================================================
# Tool dispatch helper
# ============================================================
def _dispatch_tool(raw_name, args):
    """
    Normalize the tool name, find the matching tool in the registry,
    and call the underlying Python function.

    Parameters:
        raw_name (str): Tool name from the model (may vary in case)
        args (dict): Arguments to pass to the tool

    Returns:
        Any: The result of the tool execution, or an error message if not found
    """
    # Create a lowercase mapping of tool names for case-insensitive matching
    name_map = {k.lower(): k for k in tool_registry.keys()}

    # Check if the normalized name exists in the registry
    if raw_name in name_map:
        tool_key = name_map[raw_name]
        # Call the original Python function stored in the @tool wrapper
        return tool_registry[tool_key].func(**args)

    # Return an error if the tool name is not recognized
    return f"Tool '{raw_name}' not recognized."


# ============================================================
# Function call handler
# ============================================================
def execute_function_from_response(model_response):
    """
    Check for a function call in the model's response and execute the matching tool.

    Supports:
        1. Native function_call objects (from APIs like OpenAI or Anthropic)
        2. JSON-formatted text output (for backends like Ollama)

    Parameters:
        model_response: The model's output object

    Returns:
        Any: Tool execution result or plain text response
    """
    # 1. Native function_call branch
    function_call = model_response.additional_kwargs.get("function_call")
    if function_call:
        name = function_call["name"].strip().lower()
        args = json.loads(function_call["arguments"])
        return _dispatch_tool(name, args)

    # 2. Fallback: parse JSON from text output
    try:
        # Attempt to parse the model's content as JSON
        data = json.loads(model_response.content)

        # Extract tool name and arguments from the parsed JSON
        name = data.get("name", "").strip().lower()
        args = data.get("arguments", {})

        # Dispatch to the correct tool
        return _dispatch_tool(name, args)

    except json.JSONDecodeError:
        # If parsing fails, no tool call was detected
        print("No tools detected; returning plain text response.")
        return model_response.content

# ============================================================
# System prompt
# ============================================================
system_prompt = system_message(
    "You are a helpful assistant that can use tools to perform calculations.\n"
    "Available tools:\n"
    "1. add_numbers(a, b) — Return the sum of two numbers.\n"
    "2. subtract_numbers(a, b) — Return the difference between two numbers.\n"
    "3. multiply_numbers(a, b) — Return the product of two numbers.\n"
    "If a user's question requires math, respond with either:\n"
    " - A native function_call (if supported by your system), OR\n"
    " - A JSON object in this format:\n"
    '{{"name": "<exact_tool_name>", "arguments": {{"a": <number>, "b": <number>}}}}\n'
    "If no calculation is needed, respond normally in plain text."
)


# ============================================================
# Example user questions
# ============================================================
user_questions = [
    "What is 15 minus 7?",   # Should trigger subtract_numbers
    "Tell me a joke.",       # Should return plain text
    "What is 13 times 32?"   # Should trigger multiply_numbers
]


# ============================================================
# Process each question
# ============================================================
for question in user_questions:
    # Build the message sequence for this question
    message_sequence = [system_prompt, user_message(question)]

    # Create the conversation chain and append the function call handler
    math_chain = create_conversation_chain(message_sequence) | RunnableLambda(execute_function_from_response)

    # Invoke the chain and print the result
    result = math_chain.invoke({})
    print(f"Q: {question}\nA: {result}\n")

Q: What is 15 minus 7?
A: 8

No tools detected; returning plain text response.
Q: Tell me a joke.
A: Here's one:

Why couldn't the bicycle stand up by itself?

(Wait for it...)

Because it was two-tired!

Hope that made you smile!

Q: What is 13 times 32?
A: 416

